# Start code

In [43]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import models, transforms
import torchvision.models as models
from torchvision.models import ResNet50_Weights
from sklearn.model_selection import train_test_split
from torch.optim.lr_scheduler import ReduceLROnPlateau

import json
import cv2
import numpy as np
import os

In [44]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(torch.cuda.is_available())

False


# Create Torch Dataset

In [45]:
class KeypointsDataset(Dataset):
    def __init__(self, img_dir, data_file):
        self.img_dir = img_dir
        with open(data_file, "r") as f:
            self.data = json.load(f)

        self.transforms = transforms.Compose([
            transforms.ToPILImage(),
            transforms.Resize((224, 224)),
            transforms.Grayscale(num_output_channels=3),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        img = cv2.imread(f"{self.img_dir}/{item['id']}.png")
        h,w = img.shape[:2]

        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        img = self.transforms(img)
        kps = np.array(item['kps']).flatten()
        kps = kps.astype(np.float32)

        kps[::2] *= 224.0 / w # Adjust x coordinates
        kps[1::2] *= 224.0 / h # Adjust y coordinates

        return img, kps

In [46]:
# Split data
def split_data(image_folder, annotations_path, train_ratio=0.8):
    with open(annotations_path, 'r') as f:
        annotations = json.load(f)

    # Split data into train and validation sets
    train_annotations, val_annotations = train_test_split(annotations, train_size=train_ratio, random_state=42)

    # Save the splits into separate JSON files
    train_annotations_path = os.path.join(image_folder, '../data_train.json')
    val_annotations_path = os.path.join(image_folder, '../data_val.json')

    with open(train_annotations_path, 'w') as f:
        json.dump(train_annotations, f, indent=4)

    with open(val_annotations_path, 'w') as f:
        json.dump(val_annotations, f, indent=4)

    print(f"Train annotations saved to: {train_annotations_path}")
    print(f"Validation annotations saved to: {val_annotations_path}")

In [47]:
image_folder = "training/own_dataset_6kp"
annotations_path = "training/own_dataset_6kp/annotations.json"

In [48]:
split_data(image_folder, annotations_path)

Train annotations saved to: training/own_dataset_6kp\../data_train.json
Validation annotations saved to: training/own_dataset_6kp\../data_val.json


In [49]:
train_dataset = KeypointsDataset("training/own_dataset_6kp", "training/own_dataset_6kp\../data_train.json")
val_dataset = KeypointsDataset("training/own_dataset_6kp", "training/own_dataset_6kp\../data_val.json")

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=True)


# Creat model pre-load resnet

In [50]:
# Define the path to the downloaded weights file
weights_path = 'models/resnet50-0676ba61.pth'

# # Load the model without pre-trained weights
# model = models.resnet50(weights=ResNet50_Weights.IMAGENET1K_V1)

# Load the pre-trained weights from the file with map_location to handle CPU-only environments
model = models.resnet50()
model.load_state_dict(torch.load(weights_path, map_location=device))

# Modify the fully connected layer to match the number of keypoints
num_keypoints = 6  # Example number of keypoints
model.fc = nn.Linear(model.fc.in_features, num_keypoints * 2)  # For (x, y) coordinates

# Move the model to the appropriate device
model = model.to(device)



# Create model from scratch (Hourglass)

In [55]:
# Function to create DataLoader instances
def create_dataloaders(data_folder_path, batch_size=8):
    transform = transforms.Compose([
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(10),
        transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),
        transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),
        transforms.ToTensor()
    ])

    train_dataset = KeypointsDataset(os.path.join(data_folder_path, 'images'), '/kaggle/working/data_train.json', transform=transform)
    val_dataset = KeypointsDataset(os.path.join(data_folder_path, 'images'), '/kaggle/working/data_val.json', transform=transforms.ToTensor())

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

    return train_loader, val_loader

# Train model

In [56]:
criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

In [54]:
epochs=10
for epoch in range(epochs):
    for i, (imgs,kps) in enumerate(train_loader):
        imgs = imgs.to(device)
        kps = kps.to(device)
        optimizer.zero_grad()
        outputs = model(imgs)
        loss = criterion(outputs, kps)
        loss.backward()
        optimizer.step()

        if i % 10 == 0:
            print(f"Epoch {epoch}, iter {i}, loss: {loss.item()}")

Epoch 0, iter 0, loss: 16095.8134765625


KeyboardInterrupt: 

In [ ]:
torch.save(model.state_dict(), "keypoints_model_weights.pth")